In [ ]:
!pwd

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd

mpl.rcParams['figure.dpi'] = 300

from IPython.display import Image

In [ ]:
import seaborn as sns
print(sns.__version__)

### Bug in FacetGrid in Seaborn 0.11.1
https://github.com/mwaskom/seaborn/commit/dc10c4c55ae1e52beef2cb161b91b26ef280e14a

https://github.com/mwaskom/seaborn/issues/1560

Unrelated to actual issue, https://github.com/mwaskom/seaborn/issues/1653

## Batch size = 1 (mean runtimes only; no error bars)

### Runtime growth w.r.t. input sequence length

In [ ]:
# data = np.loadtxt('results.txt',skiprows=1)
data = pd.read_csv('results-bs1.csv')

In [ ]:
data.columns

In [ ]:
data['Average time (ms)'] = data['Average time (s)']*1000

In [ ]:
# reverse order of legend
hue_order = data['Hidden size of each LSTM layer'].unique()

In [ ]:
hue_order[::-1].sort()
# print(type(hue_order))
# print(hue_order.dtype)
# hue_order = np.array(map(str, hue_order))

#### KGF: I guess we dont need str keys for hue_order, style_order to match pandas "category" dtype?
# hue_order = hue_order.astype(str)
print(hue_order)

In [ ]:
data['Hidden size of each LSTM layer'] = data['Hidden size of each LSTM layer'].astype('category')
print(data['Hidden size of each LSTM layer'])

In [ ]:
# fig, ax = plt.subplots()
# ax = sns.lineplot(data=data, x='Length of input sequences', y='Average time (ms)', style='Num LSTM layers', hue='Hidden size of each LSTM layer', ci=None)  # , marker='o', dashes=False);
# ax.set_ylabel('Average inference time (ms)')
# ax.set_title("A100 inference time for batch size=1")

g = sns.relplot(data=data, x='Length of input sequences', y='Average time (ms)', 
                 style='Hidden size of each LSTM layer', 
                 hue='Hidden size of each LSTM layer', 
                 col='Num LSTM layers', ci=None, kind='line', 
                 hue_order=hue_order, style_order=hue_order,
                  # https://github.com/mwaskom/seaborn/issues/1653
                 legend='full',
                 dashes=False,
                 col_wrap=2, markers=True) # marker=['o']) 

# reverse order of legend
#handles, labels = g.legend.legendHandles


# https://stackoverflow.com/questions/68591271/how-can-i-combine-hue-and-style-groups-in-a-seaborn-legend
# create a dictionary mapping the subscales to their color
# handles, labels = ax.get_legend_handles_labels()
# index_item_title = labels.index('Hidden size of each LSTM layer')
# color_dict = {label: handle.get_color()
#               for handle, label in zip(handles[1:index_item_title], labels[1:index_item_title])}

# # loop through the items, assign color via the subscale of the item idem
# for handle, label in zip(handles[index_item_title + 1:], labels[index_item_title + 1:]):
#     handle.set_color(color_dict[subscale_dict[ label]])

# # create a legend only using the items
# ax.legend(handles[index_item_title + 1:], labels[index_item_title + 1:], title='Item',
#           bbox_to_anchor=(1.03, 1.02), fontsize=10)

g.axes[1].plot([128], [0.00253291*1000], marker='*', markersize=15, markeredgecolor="red",
               markerfacecolor="red")
for ax in g.axes:
    ax.axhline(y=1, linestyle='--', color='gray', alpha=0.4 )


In [ ]:
g.legend.legendHandles

In [ ]:
g.figure.savefig("a100-batch-size-1.pdf")

In [ ]:
data['Hidden size of each LSTM layer'].unique()

In [ ]:
data[data['Num LSTM layers'] == 6]

In [ ]:
data[data['Num LSTM layers'] == 4].drop(
    # labels=[0], axis=1)
    columns=data.columns[[0, -2, -3]]).sort_values(by=['Hidden size of each LSTM layer', 'Length of input sequences'])

### Examine growth w.r.t. hidden size

In [ ]:
data = pd.read_csv('results-bs1.csv')
data['Average time (ms)'] = data['Average time (s)']*1000

In [ ]:
hue_order = data['Length of input sequences'].unique()
hue_order[::-1].sort()
data['Length of input sequences'] = data['Length of input sequences'].astype('category')
# print(data['Length of input sequences'])

In [ ]:
g = sns.relplot(data=data, x='Hidden size of each LSTM layer', y='Average time (ms)', 
                 style='Length of input sequences', 
                 hue='Length of input sequences', 
                 col='Num LSTM layers', ci=None, kind='line', 
                 hue_order=hue_order, style_order=hue_order,
                 legend='full',
                 dashes=False,
                 col_wrap=2, markers=True) 

g.axes[1].plot([200], [0.00253291*1000], marker='*', markersize=15, markeredgecolor="red",
               markerfacecolor="red")
for ax in g.axes:
    ax.axhline(y=1, linestyle='--', color='gray', alpha=0.4 )
    

In [ ]:
g.figure.savefig("a100-batch-size-1-xaxis-hidden-size.pdf")

## Batch size >= 1 (mean runtimes only; no error bars)

In [ ]:
data = pd.read_csv('results.csv')
data['Average time (ms)'] = data['Average time (s)']*1000

In [ ]:
data.columns

In [ ]:
data.head()

In [ ]:
seq_length = 64
num_layers = 2

In [ ]:
df = data[(data['Num LSTM layers'] == num_layers) & (data['Length of input sequences'] == seq_length) ]

In [ ]:
df['Batch size'].unique()

In [ ]:
hue_order = df['Hidden size of each LSTM layer'].unique()
hue_order[::-1].sort()
df['Hidden size of each LSTM layer'] = df['Hidden size of each LSTM layer'].astype('category')
print(df['Hidden size of each LSTM layer'])

In [ ]:
df.head()

In [ ]:
g = sns.relplot(data=df, x='Batch size', y='Average time (ms)', 
                 style='Hidden size of each LSTM layer', 
                 hue='Hidden size of each LSTM layer', 
                 # col='Num LSTM layers', col_wrap=2, 
                 ci=None, kind='line', 
                 hue_order=hue_order, style_order=hue_order,
                 legend='full',
                 dashes=False,
                 markers=True) 
#g.set(xscale="log")  # , basex=2)
# g.set(yscale="log")

ax =g.axes[0,0]
ax.axhline(y=1, linestyle='--', color='gray', alpha=0.4 )
ax.set_xscale('log', base=2)
# ax.set_ylim([0,10]);

g.fig.subplots_adjust(top=.93)
ax.set_title(f'{num_layers} LSTM layers; input sequence length= {seq_length}')

In [ ]:
g.figure.savefig(f"a100-layers{num_layers}-length{seq_length}.pdf")










## Batch size >= 1 (with error bars)

In [ ]:
data = pd.read_csv('results-raw.csv')
data['Inference time (ms)'] = data['Inference time (s)']*1000

### Pandas summary stats

In [ ]:
# data
data.head()

In [ ]:
# Q1, Q3 for 6 layer LSTM with 400 hidden units, batch size-512, input squence=32
# 7.258159999999999 7.3257175
inf_times = data[((data['Hidden size of each LSTM layer'] == 400) 
           & (data['Num LSTM layers'] == 6)
           & (data['Batch size']==512)
           & (data['Length of input sequences']==32)
     )]['Inference time (ms)']

get_num_outliers(inf_times)

In [ ]:
gb = data.groupby(['Hidden size of each LSTM layer','Num LSTM layers','Batch size','Length of input sequences'])


In [ ]:
gb['Inference time (ms)'].describe()

#### Count outliers
https://stackoverflow.com/questions/39068214/how-to-count-outliers-for-all-columns-in-python

https://stackoverflow.com/questions/48994157/get-the-number-of-outliers-by-group-in-pandas

https://stackoverflow.com/questions/43264095/python-seaborn-how-are-outliers-determined-in-boxplots
> It appears, by testing, that seaborn uses whis=1.5 as the default.

> For a normal distribution, the interquartile range contains 50% of the population and 1.5 * IQR contains about 99%.

In [ ]:
def get_num_outliers(column):
    q1 = np.percentile(column, 25)
    q3 = np.percentile(column, 75)
    whis = 1.5
    iqr = q3-q1
#     print(q1, q3)
    return sum((column<(q1 - whis*iqr)) | (column>(q3 + whis*iqr)))

In [ ]:
sgb = gb['Inference time (ms)']
print(type(gb), type(sgb))

In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None)


In [ ]:
gb.agg([get_num_outliers]).loc[200, 2, :, 64]

In [ ]:
pd.set_option("display.max_rows", 100) #, "display.max_columns", None)
pd.reset_option('display.max_rows')

In [ ]:
# df2 = sgb.apply(pd.DataFrame)

In [ ]:
Q1 = sgb.quantile(0.25)
Q3 = sgb.quantile(0.75)
IQR = Q3 - Q1

In [ ]:
(Q1 - 1.5 * IQR).shape

In [ ]:
df['Inference time (ms)'].shape

In [ ]:
mask = (sgb < (Q1 - 1.5 * IQR)) | (sgb > (Q3 + 1.5 * IQR))


### Plot subset (boxplot?)

In [ ]:
seq_length = 128
num_layers = 2
hidden_size = 200

In [ ]:
df = data[(data['Num LSTM layers'] == num_layers) & (data['Length of input sequences'] == seq_length) 
           & (data['Hidden size of each LSTM layer'] == hidden_size)
          #& (data['Batch size'] == 1)
          
         ]
df['Hidden size of each LSTM layer'].unique()
df.head()
pd.reset_option('display.max_rows')
df

In [ ]:
# Approach #1: seaborn


# ax = sns.boxplot(data=df, x='Batch size', y='Inference time (ms)',
#                   # hue='Hidden size of each LSTM layer',
#                  fliersize=0.5,  # 5 is default
#                  #showfliers=False,
#                 )

# ax.axhline(y=1, linestyle='--', color='gray', alpha=0.4 )

# KGF: cant use traditional x-axis changes, since boxplot/violinplot interprets x as categorical, not numerical
# ax.set_xscale('log', base=2)
#
# See ax.get_xlim()
#  > (-0.5, 9.5)


# --------------------------------------------------------------------
# Approach #2: matplotlib

# fig, ax = plt.subplots()
# boxdict = ax.boxplot(data)

# --------------------------------------------------------------------
# Approach #3: pandas
fig, ax = plt.subplots()
boxdict = df.boxplot(column='Inference time (ms)',
                     by='Batch size',
                     ax=ax,
                     return_type='dict')

# len(boxdict['Inference time (ms)']['fliers'][0].get_xdata())

# --------------------------------------------------------------------
# For approaches #2 and #3:

boxdict = boxdict['Inference time (ms)']
# print(len(boxdict['fliers']))

# loop over boxes in x direction
for j in range(len(boxdict['fliers'])):
    # the y and x positions of the fliers
    yfliers = boxdict['fliers'][j].get_ydata()
    xfliers = boxdict['fliers'][j].get_xdata()
    
    num_outliers = len(xfliers)
    ax.text(xfliers[j] + 0.03, np.max(yfliers)*1.03, num_outliers)
        
ax.set_title(f'{num_layers} LSTM layers sized {hidden_size}; input sequence length= {seq_length}')
ax.set_ylabel('Inference time (ms)')
plt.suptitle('')

In [ ]:
ax.figure.savefig(f"a100-boxplot-layers{num_layers}-length{seq_length}-size{hidden_size}.pdf")

### Violinplot

In [ ]:
df = data[(data['Num LSTM layers'] == num_layers) & (data['Length of input sequences'] == seq_length) 
           & ((data['Hidden size of each LSTM layer'] == 200) | (data['Hidden size of each LSTM layer'] == 400) 
             )
          & (data['Batch size'] == 1)
         ]
df.head()
pd.reset_option('display.max_rows')
df

In [ ]:
hue_order = df['Hidden size of each LSTM layer'].unique()
hue_order[::-1].sort()
df.loc[:,'Hidden size of each LSTM layer'] = df['Hidden size of each LSTM layer'].astype('category')
print(df['Hidden size of each LSTM layer'])

In [ ]:
ax = sns.violinplot(data=df, x='Batch size', y='Inference time (ms)',
                    hue='Hidden size of each LSTM layer', 
                    inner=None, # 'point', # 'quart',  #'box',
                    split=True,
                    #fontsize=6,
                )
ax.axhline(y=1, linestyle='--', color='gray', alpha=0.4 )
ax.set_title(f'{num_layers} LSTM layers; input sequence length= {seq_length}')

# ax.set_yscale('log')
ax.legend(title='Hidden size', prop=dict(size=8))

In [ ]:
ax.figure.savefig(f"a100-violinplot-layers{num_layers}-length{seq_length}-bs1.pdf")

### Line plot with confidence interval

Nevermind, `sns.lineplot` uses simple 95% confidence interval assuming normally distributed data, standard error probably estimated from all (x,y) samples?

`ci=95` default